In [84]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import numpy as np

Let's try and look up the available variables for the decennial census data collected for 2010, 2000, and 1990

In [16]:
# sample url: https://api.census.gov/data/1990/sf1/variables.html
urls = ["https://api.census.gov/data/2010/dec/sf1/variables.html", 
        "https://api.census.gov/data/2000/sf1/variables.html",
        "https://api.census.gov/data/1990/sf1/variables.html"
       ]

for url in urls: 
    print(url, '\n')

https://api.census.gov/data/2010/dec/sf1/variables.html 

https://api.census.gov/data/2000/sf1/variables.html 

https://api.census.gov/data/1990/sf1/variables.html 



In [19]:
#headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}

for url in urls: 
    result = requests.get(url)
    if result.status_code == 200: 
        print("Success!")


Success!
Success!
Success!


In [20]:
src_2010 = requests.get(urls[0]).content
src_2000 = requests.get(urls[1]).content
src_1990 = requests.get(urls[2]).content


In [21]:
soup_2010 = BeautifulSoup(src_2010, 'lxml')
soup_2000 = BeautifulSoup(src_2000, 'lxml')
soup_1990 = BeautifulSoup(src_1990, 'lxml')

In [27]:
table_2010 = soup_2010.find_all('table')
df_2010 = pd.read_html(str(table_2010))[0]

table_2000 = soup_2000.find_all('table')
df_2000 = pd.read_html(str(table_2000))[0]

table_1990 = soup_1990.find_all('table')
df_1990 = pd.read_html(str(table_1990))[0]

In [28]:
df_2010.head()
df_2000.head()
df_1990.head()

,Name,Label,Concept,Required,Attributes,Limit,Predicate Type,Group,Unnamed: 8
0,AIANACC,American Indian/Alaska Native Area Class Code,Geographic Traits,not required,NaN,0,string,NaN,NaN
1,AIANACE,American Indian/Alaska Native Area (Census),Geographic Traits,not required,NaN,0,int,NaN,NaN
2,AIANAFP,American Indian/Alaska Native Area (FIPS),Geographic Traits,not required,NaN,0,int,NaN,NaN
3,ANPSADPI,Geography Name,Geographic Concepts,not required,NaN,0,string,NaN,NaN
4,ANRC,Alaska Native Regional Corporation,Geographic Traits,not required,NaN,0,int,NaN,NaN


In [54]:
for thing in df_1990.loc[:, 'Name']: 
    if thing == "for": 
        print("for")
    if thing == "in": 
        print("in")

for
in


Now that we have the tables all of the possible variables for each census, let's count all of the usable ones.

In [62]:
vars_90 = df_1990[df_1990['Name'] != 'in'][df_1990['Name'] != 'for'].loc[:, 'Name']
vars_00 = df_2000[df_2000['Name'] != 'in'][df_2000['Name'] != 'for'].loc[:, 'Name']
vars_10 = df_2010[df_2010['Name'] != 'in'][df_2010['Name'] != 'for'].loc[:, 'Name']

<ipython-input-62-659902f59e5f>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vars_90 = df_1990[df_1990['Name'] != 'in'][df_1990['Name'] != 'for'].loc[:, 'Name']
<ipython-input-62-659902f59e5f>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vars_00 = df_2000[df_2000['Name'] != 'in'][df_2000['Name'] != 'for'].loc[:, 'Name']
<ipython-input-62-659902f59e5f>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  vars_10 = df_2010[df_2010['Name'] != 'in'][df_2010['Name'] != 'for'].loc[:, 'Name']


In [63]:
print(len(vars_90), len(vars_00), len(vars_10))

1046 8196 8999


Let's try and grab some of the data for Rhode Island! 

In [65]:
HOST = "https://api.census.gov/data" 
year = "2010" 
dataset = "dec/sf1" 


base_url = "/".join([HOST, year, dataset])

print(base_url)

https://api.census.gov/data/2010/dec/sf1


In [75]:
predicates = {} 

get_vars = vars_10[:50]
predicates["key"] = "97d54f0b2bec96ce45749d21afa620949a6cd273"
predicates["get"] = ",".join(get_vars) 
predicates["for"] = "tract" 
predicates["in"] = "state:48;county:*"

In [76]:
r = requests.get(base_url, params=predicates)

In [77]:
print(r.text)

[["AIANHH","AIHHTLI","AITSCE","ANRC","BLKGRP","BLOCK","CBSA","CD111","CNECTA","CONCIT","COUNTY","COUSUB","CSA","DIVISION","GEO_ID","GEOCOMP","GEOVARIANT","H001001","H002001","H002002","H002003","H002004","H002005","H002006","H003001","H003002","H003003","H004001","H004002","H004003","H004004","H005001","H005002","H005003","H005004","H005005","H005006","H005007","H005008","H006001","H006002","H006003","H006004","H006005","H006006","H006007","H006008","H007001","H007002","H007003","state","county","tract"],
[null,null,null,null,null,null,"21340",null,null,null,"141",null,"999","7","1400000US48141000101","00","00","2713","2713","2713","2713","0","0","0","2713","2620","93","2620","1150","592","878","93","24","3","31","3","6","0","26","2620","1926","244","19","85","5","242","99","2620","1221","859","48","141","000101"],
[null,null,null,null,null,null,"21340",null,null,null,"141",null,"999","7","1400000US48141000111","00","00","1198","1198","1198","1198","0","0","0","1198","1152","46","1152"

In [78]:
names = r.json()[0]
data = r.json()[1:]
print(names)
df = pd.DataFrame(data=data, columns=names)

['AIANHH', 'AIHHTLI', 'AITSCE', 'ANRC', 'BLKGRP', 'BLOCK', 'CBSA', 'CD111', 'CNECTA', 'CONCIT', 'COUNTY', 'COUSUB', 'CSA', 'DIVISION', 'GEO_ID', 'GEOCOMP', 'GEOVARIANT', 'H001001', 'H002001', 'H002002', 'H002003', 'H002004', 'H002005', 'H002006', 'H003001', 'H003002', 'H003003', 'H004001', 'H004002', 'H004003', 'H004004', 'H005001', 'H005002', 'H005003', 'H005004', 'H005005', 'H005006', 'H005007', 'H005008', 'H006001', 'H006002', 'H006003', 'H006004', 'H006005', 'H006006', 'H006007', 'H006008', 'H007001', 'H007002', 'H007003', 'state', 'county', 'tract']


In [79]:
df.head()

,AIANHH,AIHHTLI,AITSCE,ANRC,BLKGRP,BLOCK,CBSA,CD111,CNECTA,CONCIT,...,H006005,H006006,H006007,H006008,H007001,H007002,H007003,state,county,tract
0,None,None,None,None,None,None,21340,None,None,None,...,85,5,242,99,2620,1221,859,48,141,000101
1,None,None,None,None,None,None,21340,None,None,None,...,39,3,56,53,1152,766,540,48,141,000111
2,None,None,None,None,None,None,21340,None,None,None,...,35,3,234,45,1578,565,408,48,141,000205
3,None,None,None,None,None,None,21340,None,None,None,...,15,5,119,20,1056,371,279,48,141,000208
4,None,None,None,None,None,None,21340,None,None,None,...,20,6,324,69,2200,481,337,48,141,000301


Okay, so it looks like we are limited to 50 variables at a time. Let's try and build up a CSV for just Rhode Island

In [89]:
N = len(vars_10)
slices = list(np.arange(0, N, 50))
slices.append(N)
print(N, '\n')
print(slices)

8999 

[0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350, 1400, 1450, 1500, 1550, 1600, 1650, 1700, 1750, 1800, 1850, 1900, 1950, 2000, 2050, 2100, 2150, 2200, 2250, 2300, 2350, 2400, 2450, 2500, 2550, 2600, 2650, 2700, 2750, 2800, 2850, 2900, 2950, 3000, 3050, 3100, 3150, 3200, 3250, 3300, 3350, 3400, 3450, 3500, 3550, 3600, 3650, 3700, 3750, 3800, 3850, 3900, 3950, 4000, 4050, 4100, 4150, 4200, 4250, 4300, 4350, 4400, 4450, 4500, 4550, 4600, 4650, 4700, 4750, 4800, 4850, 4900, 4950, 5000, 5050, 5100, 5150, 5200, 5250, 5300, 5350, 5400, 5450, 5500, 5550, 5600, 5650, 5700, 5750, 5800, 5850, 5900, 5950, 6000, 6050, 6100, 6150, 6200, 6250, 6300, 6350, 6400, 6450, 6500, 6550, 6600, 6650, 6700, 6750, 6800, 6850, 6900, 6950, 7000, 7050, 7100, 7150, 7200, 7250, 7300, 7350, 7400, 7450, 7500, 7550, 7600, 7650, 7700, 7750, 7800, 7850, 7900, 7950, 8000, 8050, 8100, 8150, 8200, 8250, 8300, 8350, 8400, 8

Let's try to collect two slices and concatenate the data

In [92]:
# Note Rhode Island is 44
predicates = {} 
predicates["key"] = "97d54f0b2bec96ce45749d21afa620949a6cd273"
predicates["for"] = "tract" 
predicates["in"] = "state:44;county:*"

get_vars = vars_10[slices[0]:slices[1]]
predicates["get"] = ",".join(get_vars) 

r = requests.get(base_url, params=predicates)
names = r.json()[0]
data = r.json()[1:]
df_0 = pd.DataFrame(data=data, columns=names)

get_vars = vars_10[slices[1]:slices[2]]
predicates["get"] = ",".join(get_vars) 
r = requests.get(base_url, params=predicates)
names = r.json()[0]
data = r.json()[1:]
df_1 = pd.DataFrame(data=data, columns=names)

In [93]:
df_0.head()

,AIANHH,AIHHTLI,AITSCE,ANRC,BLKGRP,BLOCK,CBSA,CD111,CNECTA,CONCIT,...,H006005,H006006,H006007,H006008,H007001,H007002,H007003,state,county,tract
0,None,None,None,None,None,None,39300,None,None,None,...,5,0,3,7,1778,1766,1743,44,007,013201
1,None,None,None,None,None,None,39300,None,None,None,...,12,0,4,5,2167,2159,2128,44,007,013202
2,None,None,None,None,None,None,39300,None,None,None,...,8,0,3,11,1707,1697,1668,44,007,013300
3,None,None,None,None,None,None,39300,None,None,None,...,29,0,36,28,1711,1634,1551,44,007,013400
4,None,None,None,None,None,None,39300,None,None,None,...,77,0,88,50,1802,1616,1337,44,007,013500


In [94]:
df_1.head()

,H007004,H007005,H007006,H007007,H007008,H007009,H007010,H007011,H007012,H007013,...,H011A003,H011A004,H011B001,H011B002,H011B003,H011B004,H011C001,state,county,tract
0,8,4,5,0,1,5,12,8,0,0,...,835,613,23,18,0,5,8,44,007,013201
1,7,4,12,0,3,5,8,7,0,0,...,1017,523,19,18,0,1,12,44,007,013202
2,5,5,8,0,0,11,10,7,0,0,...,746,417,20,19,0,1,15,44,007,013300
3,29,3,29,0,4,18,77,28,4,3,...,458,892,73,43,0,30,15,44,007,013400
4,148,9,76,0,9,37,186,80,13,0,...,355,1178,417,209,11,197,21,44,007,013500


In [110]:
df_rhode_island = pd.DataFrame() 
cols_0 = df_0.columns 
cols_1 = df_1.columns 
for col in cols_0: 
    df_rhode_island[col] = df_0[col]

print(df_rhode_island.head())

for col in cols_1: 
    if col not in df_rhode_island.columns: 
        df_rhode_island[col] = df_1[col]
        
print('\n', df_rhode_island.head())

  AIANHH AIHHTLI AITSCE  ANRC BLKGRP BLOCK   CBSA CD111 CNECTA CONCIT  ...  \
0   None    None   None  None   None  None  39300  None   None   None  ...   
1   None    None   None  None   None  None  39300  None   None   None  ...   
2   None    None   None  None   None  None  39300  None   None   None  ...   
3   None    None   None  None   None  None  39300  None   None   None  ...   
4   None    None   None  None   None  None  39300  None   None   None  ...   

  H006005 H006006 H006007 H006008 H007001 H007002 H007003 state county   tract  
0       5       0       3       7    1778    1766    1743    44    007  013201  
1      12       0       4       5    2167    2159    2128    44    007  013202  
2       8       0       3      11    1707    1697    1668    44    007  013300  
3      29       0      36      28    1711    1634    1551    44    007  013400  
4      77       0      88      50    1802    1616    1337    44    007  013500  

[5 rows x 53 columns]

   AIANHH AIHHTLI AIT

Okay, so clearly that will work. Let's loop through them all and build up the total dataframe

In [114]:
from tqdm.notebook import tqdm

In [117]:
predicates = {} 
predicates["key"] = "97d54f0b2bec96ce45749d21afa620949a6cd273"
predicates["for"] = "tract" 
predicates["in"] = "state:44;county:*"

df_rhode_island = pd.DataFrame() 

for i in range(1, len(slices)): 
    print(i/len(slices)*100)
    get_vars = vars_10[slices[i-1]:slices[i]]
    predicates["get"] = ",".join(get_vars) 

    r = requests.get(base_url, params=predicates)
    names = r.json()[0]
    data = r.json()[1:]
    df = pd.DataFrame(data=data, columns=names)
    
    for col in df.columns: 
        if col not in df_rhode_island.columns: 
            df_rhode_island[col] = df[col]

0.5524861878453038
1.1049723756906076
1.6574585635359116
2.209944751381215
2.7624309392265194
3.314917127071823
3.867403314917127
4.41988950276243
4.972375690607735
5.524861878453039
6.077348066298343
6.629834254143646
7.18232044198895
7.734806629834254
8.287292817679557
8.83977900552486
9.392265193370166
9.94475138121547
10.497237569060774
11.049723756906078
11.602209944751381
12.154696132596685
12.70718232044199
13.259668508287293
13.812154696132598
14.3646408839779
14.917127071823206
15.469613259668508
16.022099447513813
16.574585635359114
17.12707182320442
17.67955801104972
18.23204419889503
18.784530386740332
19.337016574585636
19.88950276243094
20.441988950276244
20.994475138121548
21.54696132596685
22.099447513812155
22.65193370165746
23.204419889502763
23.756906077348066
24.30939226519337
24.861878453038674
25.41436464088398
25.96685082872928
26.519337016574585
27.071823204419886
27.624309392265197
28.176795580110497
28.7292817679558
29.2817679558011
29.83425414364641
30.386740

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [118]:
print(len(df_rhode_island.columns))

8953


In [119]:
df_rhode_island.to_csv("RI_census_2010.csv")

In [120]:
df_rhode_island.describe()

,AIANHH,AIHHTLI,AITSCE,ANRC,BLKGRP,BLOCK,CBSA,CD111,CNECTA,CONCIT,...,PCT023008,PCT023009,PCT023010,PCT023011,PCT023012,PCT023013,PCT023014,PCT023015,PCT023016,PCT023017
count,0,0,0,0,0,0,244,0,0,0,...,244,244,244,244,244,244,244,244,244,244
unique,0,0,0,0,0,0,1,0,0,0,...,74,41,16,35,30,92,14,8,12,5
top,NaN,NaN,NaN,NaN,NaN,NaN,39300,NaN,NaN,NaN,...,9,3,0,0,0,1,0,0,0,0
freq,NaN,NaN,NaN,NaN,NaN,NaN,244,NaN,NaN,NaN,...,13,19,86,38,57,12,91,193,163,223
